# dbApps09a Task: SQL Injection & Parameterized Queries

**Course:** Database Applications Development (145085)  
**Institution:** Medina County Career Center

---

## Overview

In this lesson, you will:
- Understand the dangers of SQL injection attacks
- Learn how to write vulnerable code and why it fails
- Write safe, parameterized queries that protect your database
- Adopt best practices for database security

## Setup: Import Libraries & Create Database

In [ ]:
import sqlite3
import pandas as pd

# Create connection to task09a database
conn = sqlite3.connect(":memory:")

# Enable foreign keys
conn.execute("PRAGMA foreign_keys = ON")
conn.commit()

print("Database connection established.")

## Create & Populate Products Table

In [ ]:
# Create the products table
createProductsTable = """
CREATE TABLE products (
    productId INTEGER PRIMARY KEY,
    productName TEXT NOT NULL,
    category TEXT NOT NULL,
    price REAL NOT NULL
)
"""

conn.execute(createProductsTable)
conn.commit()

print("Products table created.")

In [ ]:
# Insert sample product data
sampleProducts = [
    (1, "Laptop", "Electronics", 899.99),
    (2, "Mouse", "Electronics", 29.99),
    (3, "Desk Chair", "Furniture", 199.99),
    (4, "USB Cable", "Electronics", 9.99),
    (5, "Monitor", "Electronics", 299.99),
    (6, "Bookshelf", "Furniture", 149.99),
    (7, "Keyboard", "Electronics", 79.99),
    (8, "Desk Lamp", "Furniture", 49.99)
]

# Insert each product
for productId, productName, category, price in sampleProducts:
    conn.execute(
        "INSERT INTO products (productId, productName, category, price) VALUES (?, ?, ?, ?)",
        (productId, productName, category, price)
    )

conn.commit()

print(f"Inserted {len(sampleProducts)} sample products.")

In [ ]:
# Display the products table
dfProducts = pd.read_sql("SELECT * FROM products ORDER BY productId", conn)
print("\nProducts Table:")
print(dfProducts)

---

## Task 1: Write a VULNERABLE Query Using String Concatenation

### Instructions:
Below is an example of a **dangerous** way to search for products by category using an f-string.  
This code **WORKS** when the user enters normal input like "Electronics", but it opens the door to SQL injection attacks.

**Your Task:** Write this vulnerable query and test it with category = "Electronics".

In [ ]:
# VULNERABLE: String concatenation (f-string)
# WARNING: Never do this in production code!

# Store the user input (simulating form input)
userCategory = "Electronics"

# Dangerously concatenate the input into the SQL query
vulnerableQuery = f"SELECT * FROM products WHERE category = '{userCategory}'"

print("Vulnerable Query:")
print(vulnerableQuery)
print()

# Execute the query (this works fine with normal input)
dfVulnerable = pd.read_sql(vulnerableQuery, conn)
print("Results:")
print(dfVulnerable)

---

## Task 2: Demonstrate SQL Injection - OR Clause Attack

### The Attack:
What if a malicious user entered `' OR '1'='1` as the category?

**Write out the resulting SQL string below:**

```
SELECT * FROM products WHERE category = '' OR '1'='1'
```

**Why is this dangerous?**
- The condition `'1'='1'` is always TRUE
- The query becomes: SELECT * FROM products WHERE category = '' OR TRUE
- This means: Return all rows, regardless of category
- An attacker could see data they shouldn't have access to

**What happens in code:**

In [ ]:
# DEMONSTRATION ONLY: Showing what the SQL looks like (not executing destructive SQL)

# Simulate a malicious input
maliciousCategory = "' OR '1'='1"

# Show what the query becomes
injectionQuery = f"SELECT * FROM products WHERE category = '{maliciousCategory}'"

print("If we concatenated the malicious input, the SQL would become:")
print()
print(injectionQuery)
print()
print("Notice: The WHERE clause now contains an always-true condition (OR '1'='1')")
print("This would return ALL products, not just one category.")
print()
print("This is a SQL injection attack!")

---

## Task 3: Demonstrate SQL Injection - Delete Attack

### The Attack:
What if a malicious user entered `'; DELETE FROM products; --` as the category?

**Write out the resulting SQL string below:**

```
SELECT * FROM products WHERE category = ''; DELETE FROM products; --'
```

**Why is this catastrophic?**
- The semicolon ends the SELECT statement
- A DELETE command is injected
- The `--` comments out the rest (the trailing `'`)
- Result: The entire products table is deleted!

**What happens in code:**

In [ ]:
# DEMONSTRATION ONLY: Showing what the SQL looks like (NOT executing it)

# Simulate a malicious input that would delete the table
maliciousDeleteCategory = "'; DELETE FROM products; --"

# Show what the query becomes
injectionDeleteQuery = f"SELECT * FROM products WHERE category = '{maliciousDeleteCategory}'"

print("If we concatenated this malicious input, the SQL would become:")
print()
print(injectionDeleteQuery)
print()
print("Breaking it down:")
print("  1. SELECT * FROM products WHERE category = ''   <- Returns empty results")
print("  2. DELETE FROM products;                         <- DELETES ALL ROWS!")
print("  3. --'                                           <- Commented out (ignored)")
print()
print("RESULT: The entire products table would be destroyed!")
print()
print("We are NOT executing this malicious query.")
print("In production, this would be a critical data loss incident.")

---

## Task 4: Write a SAFE Query Using Parameterized Queries

### Instructions:
Rewrite the search query using **parameterized queries** (also called prepared statements).  
Use the `?` placeholder for user input. Test with category = "Electronics".

In [ ]:
# SAFE: Parameterized query using ? placeholder
# The input is passed SEPARATELY from the SQL structure

# User input (simulated from a form)
userCategory = "Electronics"

# Safe parameterized query
# The ? is a placeholder that will be safely filled by the database driver
safeQuery = "SELECT * FROM products WHERE category = ?"

print("Safe Parameterized Query:")
print(safeQuery)
print()
print(f"Parameter: ('{userCategory}',)")
print()

# Execute with parameters passed separately
dfSafe = pd.read_sql(safeQuery, conn, params=(userCategory,))
print("Results:")
print(dfSafe)

### Why is this safe?
- The SQL structure is defined FIRST
- User input is passed SEPARATELY as a parameter
- The database driver knows the difference between SQL code and data
- Even if the user enters `' OR '1'='1`, it will be treated as literal text, not SQL code
- **The input cannot change the structure of the query**

---

## Task 5: Parameterized Query - Find Products Below a Price

### Instructions:
Write a parameterized query to find all products with a price less than a given amount.  
Test with maxPrice = 100.

In [ ]:
# Find all products cheaper than a given price
maxPrice = 100

# Write your parameterized query here
queryByPrice = "SELECT * FROM products WHERE price < ? ORDER BY price"

# Execute the query safely
dfByPrice = pd.read_sql(queryByPrice, conn, params=(maxPrice,))

print(f"Products under ${maxPrice}:")
print(dfByPrice)

---

## Task 6: Parameterized Query - Search by Product Name (LIKE)

### Instructions:
Write a parameterized query to find products where the name contains a search term.  
Use SQL LIKE with % wildcards.  
Test with searchTerm = "Cable" (should find "USB Cable").

In [ ]:
# Find products by partial name match
searchTerm = "Cable"

# Write your parameterized query with LIKE and % wildcards
# Note: Build the pattern string first, then pass it as a parameter
searchPattern = f"%{searchTerm}%"

queryByName = "SELECT * FROM products WHERE productName LIKE ? ORDER BY productName"

# Execute the query safely
dfByName = pd.read_sql(queryByName, conn, params=(searchPattern,))

print(f"Products containing '{searchTerm}':")
print(dfByName)

---

## Task 7: Create Customers Table & Parameterized INSERT

### Instructions:
1. Create a "customers" table with fields: customerId, firstName, lastName, email, phone
2. Write a parameterized INSERT to add a new customer
3. Add 2-3 sample customers

In [ ]:
# Create the customers table
createCustomersTable = """
CREATE TABLE customers (
    customerId INTEGER PRIMARY KEY,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    email TEXT NOT NULL,
    phone TEXT
)
"""

conn.execute(createCustomersTable)
conn.commit()

print("Customers table created.")

In [ ]:
# Safe parameterized INSERT for a new customer
# Each ? is a placeholder for a parameter value

insertCustomerQuery = """
INSERT INTO customers (customerId, firstName, lastName, email, phone)
VALUES (?, ?, ?, ?, ?)
"""

# Add sample customers
sampleCustomers = [
    (1, "Alice", "Johnson", "alice@example.com", "555-0001"),
    (2, "Bob", "Smith", "bob@example.com", "555-0002"),
    (3, "Carol", "Williams", "carol@example.com", "555-0003")
]

# Insert each customer using the parameterized query
for customerId, firstName, lastName, email, phone in sampleCustomers:
    conn.execute(insertCustomerQuery, (customerId, firstName, lastName, email, phone))

conn.commit()

print(f"Inserted {len(sampleCustomers)} customers.")

In [ ]:
# Verify the customers were inserted
dfCustomers = pd.read_sql("SELECT * FROM customers ORDER BY customerId", conn)
print("\nCustomers Table:")
print(dfCustomers)

---

## Task 8: Parameterized Query - Safe UPDATE

### Instructions:
Write a parameterized UPDATE query to change a product's price.  
Example: Update product with productId = 5 to price = 349.99

In [ ]:
# Show current product 5 (Monitor)
print("Before update:")
dfBefore = pd.read_sql("SELECT * FROM products WHERE productId = 5", conn)
print(dfBefore)
print()

In [ ]:
# Safe parameterized UPDATE
updateProductQuery = "UPDATE products SET price = ? WHERE productId = ?"

# Parameters: new price, productId
newPrice = 349.99
productIdToUpdate = 5

# Execute the parameterized update
conn.execute(updateProductQuery, (newPrice, productIdToUpdate))
conn.commit()

print(f"Updated product {productIdToUpdate} to price ${newPrice}")
print()

In [ ]:
# Verify the update
print("After update:")
dfAfter = pd.read_sql("SELECT * FROM products WHERE productId = 5", conn)
print(dfAfter)

---

## Task 9: Best Practices for Preventing SQL Injection

### List 3 Best Practices:

1. **Always Use Parameterized Queries (Prepared Statements)**
   - Separate SQL structure from user input
   - Use ? placeholders in SQLite, named parameters in others
   - The database driver handles escaping and validation

2. **Never Concatenate or Interpolate User Input**
   - Avoid f-strings: `f"SELECT * FROM users WHERE id = {userInput}"`
   - Avoid format(): `"SELECT * FROM users WHERE id = {}".format(userInput)`
   - Avoid string concatenation: `"SELECT * FROM users WHERE id = '" + userInput + "'"`

3. **Validate and Sanitize Input**
   - Check data types (is it a number when it should be?)
   - Check length and format (email, phone, etc.)
   - Use whitelist validation (allow only known good values)
   - Even with parameterized queries, validation adds a layer of defense

---

### Bonus Practices:
- Use stored procedures in enterprise databases (less applicable in SQLite)
- Apply principle of least privilege (users have only necessary permissions)
- Keep your database framework and libraries updated
- Log and monitor SQL queries for suspicious patterns

---

## Task 10: Reflection - String Concatenation Dangers

### Question:
Why is string concatenation dangerous **even when YOU control the input**?

### Answer:

1. **Future Developers**
   - You might leave the company or move to another project
   - Another developer might copy this code pattern elsewhere
   - They might use it with untrusted user input without realizing the danger

2. **Code Reuse & Maintenance**
   - Code gets refactored, copied, and pasted
   - A function that currently takes only hardcoded values might be modified later to accept user input
   - If the foundation is unsafe, the change becomes dangerous

3. **Habit Formation**
   - Writing unsafe code creates bad habits
   - When you encounter a real user input scenario, you might use the same pattern without thinking
   - Muscle memory defaults to the "easy" approach, which is the dangerous one

4. **Copy-Paste to Other Projects**
   - You might use this code as a template for another project
   - The next time, the input might come from an API, user form, or file
   - The vulnerable code pattern is already embedded

### The Solution:
**Always code defensively. Use parameterized queries from the start, even for hardcoded values.**
- It's not much harder than string concatenation
- It protects you against future mistakes
- It sets a good example for other developers
- It's the industry standard

---

## Summary

You have learned:
- How SQL injection attacks work and why they are dangerous
- The difference between vulnerable string concatenation and safe parameterized queries
- How to write parameterized queries for SELECT, INSERT, and UPDATE operations
- Best practices to prevent SQL injection in your code

**Remember:** Always use parameterized queries. They are simple, safe, and standard across all databases.